## **Управление предприятием по переработке руды** ##

Перерабатываются два вида руды: А и В. Предприятию может быть поставлено в день до 100 тыс. тонн руды вида А по цене 3.25 руб./т. и до 30 тыс. тонн руды вида В более по цене 3.40 руб./т.  Общая мощность основного процесса переработки равна 100 тыс. тонн руды в день при затратах на переработку 0.35 руб./т. 

Основной процесс обработки позволяет получать 
из каждой тонны руды вида А 0.15 т продукта I и 0.85 т продукта II,  
из каждой тонны руды вида В – 0.25 т продукта I и 0.75 т продукта II. 
Продукт I более ценный, и агрегат, называемый конвертером, способен из каждой тонны продукта II получить 0.5 т продукта I и 0.5 т продукта II, который нельзя повторно перерабатывать конвертером. Мощность конвертера – 50 тыс. тонн сырья в день при затратах на конвертерную обработку 0.25 руб./т сырья. 
Затраты на фильтрацию продукта I , производимую после основного процесса обработки, равны 0.10 руб./т сырья. Затраты на упаковку продукта I 0.15 руб./т сырья.

Условия реализации продукции следующие. Вся продукция идет на продажу.
Продукт II может быть реализован в неограниченном количестве по цене 3.80 руб./т. Продукт I  продается по цене 5.50 руб./т, и его можно продать по этой цене до 45 тыс. т/день. Кроме того, можно продать до 4 тыс. тонн в день по цене 5.2 руб./т и неограниченное количество продукта по заниженной цене 5.0 руб./т.
Существует контракт, согласно которому требуется поставлять потребителям не менее 40 тыс. тонн продукта I в день. Оба продукта можно при необходимости докупить: закупочная цена продукта I равна 5.75 руб./т, закупочная цена продукта II - 4.00 руб./т.

Ввести переменные, построить математическую модель и 
* найти план выпуска продукции с максимальной прибылью;
* найти минимальную цену на продукт I, при которой прибыль предприятия будет не меньше 50 тыс. руб./день;
* как изменится оптимальный план, если мощность фильтра ограничена величиной 10 тыс. тонн?
* как изменится оптимальный план производства, если фильтр выйдет из строя, и продукт I можно будет продавать только по цене 5 руб./т.?

![image.png](scheme.png)

#### Запишем математическую модель:

Введём переменные:

$II$ - продано тонн продукта II  
$I_в$ - продано тонн продукта I по завышенной цене  
$I_{ср}$ - продано тонн продукта I по обычной цене  
$I_н$ - продано тонн продукта I по заниженной цене  
$a$ - поставлено тонн руды А за день  
$b$ - оставлено тонн руды В за день  
$II_{кон}$ - отправлено тонн продукта II на конвейер  
$I_{доп}$ - докупили продукта I для контракта  
$II_{доп}$ - докупили продукта II для контракта    

Целевая функция:

$$(II * 3.8 + I_в * 5.5 + I_{ср} * 5.2 + I_н * 5) - (a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_{кон} * 0.25 + 0.1 * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_{кон} + 0.25 * b + 0.15 * a) + I_{доп} * 5.75 + II_{доп} * 4) \to \max$$

Ограничения:

$$a + b \leq 100000$$
$$II_{кон} \leq 50000$$
$$I_в + I_{ср} + I_н \geq 40000$$
$$I_в + I_{ср} + I_н = 0.15 * a + 0.25 * b + 0.5 * II_{кон} + I_{доп}$$
$$II = 0.85 * a + 0.75 * b - 0.5 * II_{кон} + II_{доп}$$

In [194]:
import gurobipy as gb      
import numpy as np

#### **1. Найти план выпуска продукции с максимальной прибылью**

In [195]:
first_problem = gb.Model('first_problem')

In [196]:
II = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II')
I_h = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=45000, name='I_h')
I_m = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=4000, name='I_m')
I_l = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_l')
a = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=100000, name='a')
b = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=30000, name='b')
II_con = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=50000, name='II_con')
I_ex = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_ex')
II_ex = first_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II_ex')

In [197]:
obj = II * 3.8 + I_h * 5.5 + I_m * 5.2 + I_l * 5
obj -= a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_con * 0.25 + 0.1 * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_con + 0.25 * b + 0.15 * a) + I_ex * 5.75 + II_ex * 4
first_problem.setObjective(obj, gb.GRB.MAXIMIZE)

In [198]:
first_problem.addConstr(a + b <= 100000)
first_problem.addConstr(II_con <= 50000)
first_problem.addConstr(I_h + I_m + I_l >= 40000)
first_problem.addConstr(I_h + I_m + I_l == 0.15 * a + 0.25 * b + 0.5 * II_con + I_ex)
first_problem.addConstr(II == 0.85 * a + 0.75 * b - 0.5 * II_con + II_ex)

first_problem.update()

In [199]:
first_problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xc2b41b5a
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 6e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [4e+04, 1e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 17 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)
Found heuristic solution: objective -13449.75000
Found heuristic solution: objective 32800.250000

Root relaxation: objective 6.800000e+04, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

In [200]:
if first_problem.status == gb.GRB.OPTIMAL:
    print(f"Поставлено {a.x} тонн руды A\n\
Поставлено {b.x} тонн руды B\n\
Ha конвейер отправлено {II_con.x} тонн продукта II\n\
Продано {I_h.x} тонн продукта I по завышенной цене\n\
Продано {I_m.x} тонн продукта I по обычной цене\n\
Продано {I_l.x} тонн продукта I по заниженной цене\n\
Продано {II.x} тонн продукта II\n\
Докупили {I_ex.x} продукта I для контракта\n\
Докупили {II_ex.x} продукта II для контракта\n\n\
Итоговая прибыль составила {first_problem.objVal}")
else:
    print('Оптимального решения нет.')

Поставлено 100000.0 тонн руды A
Поставлено 0.0 тонн руды B
Ha конвейер отправлено 50000.0 тонн продукта II
Продано 40000.0 тонн продукта I по завышенной цене
Продано -0.0 тонн продукта I по обычной цене
Продано -0.0 тонн продукта I по заниженной цене
Продано 60000.0 тонн продукта II
Докупили -0.0 продукта I для контракта
Докупили -0.0 продукта II для контракта

Итоговая прибыль составила 68000.0


#### **2. Найти минимальную цену на продукт I, при которой прибыль предприятия будет не меньше 50 тыс. руб./день**

#### Запишем математическую модель:

Введём переменные:

$I_c$ - цена продукта I  
$I$ - продано тонн продукта I

Целевая функция:

$$I_c \to \min$$

Ограничения:

$$a + b \leq 100000$$
$$II_{кон} \leq 50000$$
$$I \geq 40000$$
$$I = 0.15 * a + 0.25 * b + 0.5 * II_{кон} + I_{доп}$$
$$II = 0.85 * a + 0.75 * b - 0.5 * II_{кон} + II_{доп}$$
$$(II * 3.8 + I * I_c) - (a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_{кон} * 0.25 + 0.1 * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_{кон} + 0.25 * b + 0.15 * a) + I_{доп} * 5.75 + II_{доп} * 4) \geq 50000$$


In [201]:
second_problem = gb.Model('second_problem')

In [202]:
II = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II')
I = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I')
I_c = second_problem.addVar(vtype=gb.GRB.CONTINUOUS, lb=0.0, ub=gb.GRB.INFINITY, name='I_c')
a = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=100000, name='a')
b = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=30000, name='b')
II_con = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=50000, name='II_con')
I_ex = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_ex')
II_ex = second_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II_ex')

In [203]:
second_problem.setObjective(I_c, gb.GRB.MINIMIZE)

In [204]:
second_problem.addConstr(a + b <= 100000)
second_problem.addConstr(II_con <= 50000)
second_problem.addConstr(I >= 40000)
second_problem.addConstr(I == 0.15 * a + 0.25 * b + 0.5 * II_con + I_ex)
second_problem.addConstr(II == 0.85 * a + 0.75 * b - 0.5 * II_con + II_ex)
second_problem.addConstr((II * 3.8 + I * I_c) - (a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_con * 0.25 + 0.1 * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_con + 0.25 * b + 0.15 * a) + I_ex * 5.75 + II_ex * 4) >= 50000)

second_problem.update()

In [205]:
second_problem.setParam('NonConvex', 2)
second_problem.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 8 columns and 14 nonzeros
Model fingerprint: 0xb17811d7
Model has 1 quadratic constraint
Variable types: 1 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [3e-01, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e+04, 1e+05]
  RHS range        [4e+04, 1e+05]
  QRHS range       [5e+04, 5e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 9 columns, 26 nonzeros
Presolved model has 1 bilinear constraint(s)
Variable types: 2 continuous, 7 integer (0 binary)

Root relaxation: objective 0.000000e+00, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds

In [206]:
if second_problem.status == gb.GRB.OPTIMAL:
    print(f"Поставлено {a.x} тонн руды A\n\
Поставлено {b.x} тонн руды B\n\
Ha конвейер отправлено {II_con.x} тонн продукта II\n\
Цену на продукт I поставили {I_c.x}\n\
Продано {I_h.x} тонн продукта I\n\
Продано {II.x} тонн продукта II\n\
Докупили {I_ex.x} продукта I для контракта\n\
Докупили {II_ex.x} продукта II для контракта")
else:
    print('Оптимального решения нет.')

Поставлено 100000.0 тонн руды A
Поставлено -0.0 тонн руды B
Ha конвейер отправлено 50000.0 тонн продукта II
Цену на продукт I поставили 5.05
Продано 40000.0 тонн продукта I
Продано 60000.0 тонн продукта II
Докупили -0.0 продукта I для контракта
Докупили -0.0 продукта II для контракта


#### **3. Как изменится оптимальный план, если мощность фильтра ограничена величиной 10 тыс. тонн?**

#### Запишем математическую модель:

Введём переменные:

$II$ - продано тонн продукта II  
$I_в$ - продано тонн продукта I по завышенной цене  
$I_{ср}$ - продано тонн продукта I по обычной цене  
$I_н$ - продано тонн продукта I по заниженной цене  
$a$ - поставлено тонн руды А за день  
$b$ - оставлено тонн руды В за день  
$II_{кон}$ - отправлено тонн продукта II на конвейер  
$I_{доп}$ - докупили продукта I для контракта  
$II_{доп}$ - докупили продукта II для контракта    

Целевая функция:

$$(II * 3.8 + I_в * 5.5 + I_{ср} * 5.2 + I_н * 5) - (a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_{кон} * 0.25 + 0.1 * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_{кон} + 0.25 * b + 0.15 * a) + I_{доп} * 5.75 + II_{доп} * 4) \to \max$$

Ограничения:

$$a + b \leq 100000$$
$$II_{кон} \leq 50000$$
$$I_в + I_{ср} + I_н \geq 40000$$
$$I_в + I_{ср} + I_н = 0.15 * a + 0.25 * b + 0.5 * II_{кон} + I_{доп}$$
$$II = 0.85 * a + 0.75 * b - 0.5 * II_{кон} + II_{доп}$$
$$b * 0.25 + a * 0.15\leq 10000$$

In [207]:
third_problem = gb.Model('third_problem')

In [208]:
II = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II')
I_h = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=45000, name='I_h')
I_m = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=4000, name='I_m')
I_l = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_l')
a = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=100000, name='a')
b = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=30000, name='b')
II_con = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=50000, name='II_con')
I_ex = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_ex')
II_ex = third_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II_ex')

In [209]:
obj = II * 3.8 + I_h * 5.5 + I_m * 5.2 + I_l * 5
obj -= a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_con * 0.25 + 0.1 * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_con + 0.25 * b + 0.15 * a) + I_ex * 5.75 + II_ex * 4
third_problem.setObjective(obj, gb.GRB.MAXIMIZE)

In [210]:
third_problem.addConstr(a + b <= 100000)
third_problem.addConstr(II_con <= 50000)
third_problem.addConstr(I_h + I_m + I_l >= 40000)
third_problem.addConstr(I_h + I_m + I_l == 0.15 * a + 0.25 * b + 0.5 * II_con + I_ex)
third_problem.addConstr(II == 0.85 * a + 0.75 * b - 0.5 * II_con + II_ex)
third_problem.addConstr(b * 0.25 + a * 0.15 <= 10000)

third_problem.update()

In [211]:
third_problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 20 nonzeros
Model fingerprint: 0x393f2405
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 6e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [1e+04, 1e+05]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 17 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)
Found heuristic solution: objective -13449.75000
Found heuristic solution: objective 32800.250000

Root relaxation: objective 5.283305e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

In [212]:
if third_problem.status == gb.GRB.OPTIMAL:
    print(f"Поставлено {a.x} тонн руды A\n\
Поставлено {b.x} тонн руды B\n\
Ha конвейер отправлено {II_con.x} тонн продукта II\n\
Продано {I_h.x} тонн продукта I по завышенной цене\n\
Продано {I_m.x} тонн продукта I по обычной цене\n\
Продано {I_l.x} тонн продукта I по заниженной цене\n\
Продано {II.x} тонн продукта II\n\
Докупили {I_ex.x} продукта I для контракта\n\
Докупили {II_ex.x} продукта II для контракта\n\n\
Итоговая прибыль составила {third_problem.objVal}")
else:
    print('Оптимального решения нет.')

Поставлено 66665.0 тонн руды A
Поставлено 1.0 тонн руды B
Ha конвейер отправлено 50000.0 тонн продукта II
Продано 40000.0 тонн продукта I по завышенной цене
Продано -0.0 тонн продукта I по обычной цене
Продано -0.0 тонн продукта I по заниженной цене
Продано 31666.0 тонн продукта II
Докупили 5000.0 продукта I для контракта
Докупили -0.0 продукта II для контракта

Итоговая прибыль составила 52833.04999999999


#### **4. Как изменится оптимальный план производства, если фильтр выйдет из строя, и продукт I можно будет продавать только по цене 5 руб./т.?**

#### Запишем математическую модель:

Введём переменные:

$II$ - продано тонн продукта II  
$I$ - продано тонн продукта I   
$a$ - поставлено тонн руды А за день  
$b$ - оставлено тонн руды В за день  
$II_{кон}$ - отправлено тонн продукта II на конвейер  
$I_{доп}$ - докупили продукта I для контракта  
$II_{доп}$ - докупили продукта II для контракта    

Целевая функция:

$$(II * 3.8 + I * 5) - (a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_{кон} * 0.25 + 0.15 * (0.5 * II_{кон} + 0.25 * b + 0.15 * a) + I_{доп} * 5.75 + II_{доп} * 4) \to \max$$

Ограничения:

$$a + b \leq 100000$$
$$II_{кон} \leq 50000$$
$$I \geq 40000$$
$$I = 0.15 * a + 0.25 * b + 0.5 * II_{кон} + I_{доп}$$
$$II = 0.85 * a + 0.75 * b - 0.5 * II_{кон} + II_{доп}$$

In [213]:
fourth_problem = gb.Model('fourth_problem')

In [214]:
II = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II')
I = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_h')
a = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=100000, name='a')
b = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=30000, name='b')
II_con = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=50000, name='II_con')
I_ex = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_ex')
II_ex = fourth_problem.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II_ex')

In [215]:
obj = II * 3.8 + I * 5
obj -= a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_con * 0.25 + 0.15 * (0.5 * II_con + 0.25 * b + 0.15 * a) + I_ex * 5.75 + II_ex * 4
fourth_problem.setObjective(obj, gb.GRB.MAXIMIZE)

In [216]:
fourth_problem.addConstr(a + b <= 100000)
fourth_problem.addConstr(II_con <= 50000)
fourth_problem.addConstr(I >= 40000)
fourth_problem.addConstr(I == 0.15 * a + 0.25 * b + 0.5 * II_con + I_ex)
fourth_problem.addConstr(II == 0.85 * a + 0.75 * b - 0.5 * II_con + II_ex)

fourth_problem.update()

In [217]:
fourth_problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 7 columns and 14 nonzeros
Model fingerprint: 0xfebf2bc6
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 6e+00]
  Bounds range     [3e+04, 1e+05]
  RHS range        [4e+04, 1e+05]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 7 columns, 12 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)
Found heuristic solution: objective -30000.00000
Found heuristic solution: objective 14250.000000

Root relaxation: objective 4.950000e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

In [218]:
if fourth_problem.status == gb.GRB.OPTIMAL:
    print(f"Поставлено {a.x} тонн руды A\n\
Поставлено {b.x} тонн руды B\n\
Ha конвейер отправлено {II_con.x} тонн продукта II\n\
Продано {I.x} тонн продукта I\n\
Продано {II.x} тонн продукта II\n\
Докупили {I_ex.x} продукта I для контракта\n\
Докупили {II_ex.x} продукта II для контракта\n\n\
Итоговая прибыль составила {fourth_problem.objVal}")
else:
    print('Оптимального решения нет.')

Поставлено 100000.0 тонн руды A
Поставлено 0.0 тонн руды B
Ha конвейер отправлено 50000.0 тонн продукта II
Продано 40000.0 тонн продукта I
Продано 60000.0 тонн продукта II
Докупили -0.0 продукта I для контракта
Докупили -0.0 продукта II для контракта

Итоговая прибыль составила 49500.0


#### **Вопрос 1.** 

У предприятия по переработке руды есть несколько возможных фильтров с заданными условиями: первый фильтр с мощностью 20 тыс. тонн, причем затраты на фильтрацию составляют 0,01 руб/т, второй фильтр с мощностью 32 тыс. тонн, причем затраты на фильтрацию составляют 0,25 руб/т, и третий фильтр с мощностью 40 тыс. тонн, причем затраты на фильтрацию составляют 0,5 руб/т. Какой фильтр выгоднее всего поставить предприятию?

In [233]:
max_income = 0, 0

for i, cost, power in [(1, 0.01, 20000), (2, 0.25, 32000), (3, 0.5, 40000)]:
    first_question = gb.Model('first_question')

    II = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II')
    I_h = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=45000, name='I_h')
    I_m = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=4000, name='I_m')
    I_l = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_l')
    a = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=100000, name='a')
    b = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=30000, name='b')
    II_con = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=50000, name='II_con')
    I_ex = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_ex')
    II_ex = first_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II_ex')

    obj = II * 3.8 + I_h * 5.5 + I_m * 5.2 + I_l * 5
    obj -= a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_con * 0.25 + cost * (0.25 * b + 0.15 * a) + 0.15 * (0.5 * II_con + 0.25 * b + 0.15 * a) + I_ex * 5.75 + II_ex * 4
    first_question.setObjective(obj, gb.GRB.MAXIMIZE)

    first_question.addConstr(a + b <= 100000)
    first_question.addConstr(II_con <= 50000)
    first_question.addConstr(I_h + I_m + I_l >= 40000)
    first_question.addConstr(I_h + I_m + I_l == 0.15 * a + 0.25 * b + 0.5 * II_con + I_ex)
    first_question.addConstr(II == 0.85 * a + 0.75 * b - 0.5 * II_con + II_ex)
    first_question.addConstr(0.25 * b + 0.15 * a <= power)

    first_question.update()

    first_question.optimize()

    if first_question.objVal >= max_income[1]:
        max_income = i, first_question.ObjVal

print(f'Выгоднее всего поставить {max_income[0]} фильтр, так как прибыль составляет {max_income[1]} рублей')

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 20 nonzeros
Model fingerprint: 0x18e17b6c
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 6e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [2e+04, 1e+05]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 17 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)
Found heuristic solution: objective -13449.75000
Found heuristic solution: objective 33475.250000

Root relaxation: objective 6.947000e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

#### **Вопрос 2.**

Предприятие решилось обновиться, и теперь появился новый улучшенный фильтр у первого продукта после основного процесса переработки. После фильтра продукт I становится чище, и его стоимость возрастает на 20%, и при этом сами затраты на фильтрацию минимальны (то есть считаем, что бесплатно). Рассчитайте математическую модель с новыми условиями.

In [234]:
second_question = gb.Model('second_question')

In [235]:
II = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II')
I_h = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=45000, name='I_h')
I_m = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=4000, name='I_m')
I_l = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_l')
a = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=100000, name='a')
b = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=30000, name='b')
II_con = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=50000, name='II_con')
I_ex = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='I_ex')
II_ex = second_question.addVar(vtype=gb.GRB.INTEGER, lb=0.0, ub=gb.GRB.INFINITY, name='II_ex')

In [237]:
obj = II * 3.8 + I_h * 5.5 + I_m * 5.2 + I_l * 5 + 0.2 * (a * 0.15 + b * 0.25)
obj -= a * 3.25 + b * 3.4 + 0.35 * (a + b) + II_con * 0.25 + 0.15 * (0.5 * II_con + 0.25 * b + 0.15 * a) + I_ex * 5.75 + II_ex * 4
second_question.setObjective(obj, gb.GRB.MAXIMIZE)

In [238]:
second_question.addConstr(a + b <= 100000)
second_question.addConstr(II_con <= 50000)
second_question.addConstr(I_h + I_m + I_l >= 40000)
second_question.addConstr(I_h + I_m + I_l == 0.15 * a + 0.25 * b + 0.5 * II_con + I_ex)
second_question.addConstr(II == 0.85 * a + 0.75 * b - 0.5 * II_con + II_ex)

second_question.update()

In [239]:
second_question.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xce9169b1
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 6e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [4e+04, 1e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 17 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)
Found heuristic solution: objective -13449.75000
Found heuristic solution: objective 35050.250000

Root relaxation: objective 7.325000e+04, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0          

In [240]:
if second_question.status == gb.GRB.OPTIMAL:
    print(f"Поставлено {a.x} тонн руды A\n\
Поставлено {b.x} тонн руды B\n\
Ha конвейер отправлено {II_con.x} тонн продукта II\n\
Продано {I_h.x} тонн продукта I по завышенной цене\n\
Продано {I_m.x} тонн продукта I по обычной цене\n\
Продано {I_l.x} тонн продукта I по заниженной цене\n\
Продано {II.x} тонн продукта II\n\
Докупили {I_ex.x} продукта I для контракта\n\
Докупили {II_ex.x} продукта II для контракта\n\n\
Итоговая прибыль составила {second_question.objVal}")
else:
    print('Оптимального решения нет.')

Поставлено 70000.0 тонн руды A
Поставлено 30000.0 тонн руды B
Ha конвейер отправлено 50000.0 тонн продукта II
Продано 43000.0 тонн продукта I по завышенной цене
Продано -0.0 тонн продукта I по обычной цене
Продано -0.0 тонн продукта I по заниженной цене
Продано 57000.0 тонн продукта II
Докупили -0.0 продукта I для контракта
Докупили -0.0 продукта II для контракта

Итоговая прибыль составила 73249.99999999996
